In [1]:
import os
import numpy as np
import pydicom
import nibabel as nib
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm

In [2]:
def process_image(gray):
    # 엣지 감지 및 팽창 (Canny + Dilate)
    edges = cv2.Canny(gray, 50, 150)
    dilated = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)

    # 컨투어 검출
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 기본 마스크 초기화
    mask_1 = np.zeros_like(gray, dtype=np.uint8)  # 큰 컨투어만 담을 마스크
    final_gray = gray.copy()                      # 최종 처리된 이미지
    mask_temp=np.zeros_like(gray, dtype=np.uint8)
    if len(contours) > 1:
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        largest_contour = contours[0]
        small_contours = contours[1:]

        # 작은 컨투어 마스크
        small_mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.fillPoly(small_mask, small_contours, 255)
        final_gray[small_mask > 0] = 0  # 작은 컨투어 제거

        # 큰 컨투어만 채운 마스크
        cv2.fillPoly(mask_1, [largest_contour], 255)
        mask_temp[mask_1 > 0] = 255
    return final_gray, mask_temp
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def window_and_normalize(img, dcm):
    # HU 변환
    slope = dcm.RescaleSlope if 'RescaleSlope' in dcm else 1
    intercept = dcm.RescaleIntercept if 'RescaleIntercept' in dcm else 0
    img = img * slope + intercept

    # WL/WW 적용
    wl = dcm.WindowCenter[0] if isinstance(dcm.WindowCenter, pydicom.multival.MultiValue) else dcm.WindowCenter
    ww = dcm.WindowWidth[0] if isinstance(dcm.WindowWidth, pydicom.multival.MultiValue) else dcm.WindowWidth

    if wl is not None and ww is not None:
        min_val = wl - ww / 2
        max_val = wl + ww / 2
        img = np.clip(img, min_val, max_val)
        img = (img - min_val) / (max_val - min_val)
    else:
        # fallback
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)

    # [-1, 1] 스케일링
    img = img * 2 - 1
    return img

# ==========================
# ⚙️ 설정
# ==========================
class_list = ['Ischemic', 'Hemorrhagic', 'Normal']
save_path='../../data/stroke_ct_nii/'
for i in range(len(class_list)):
    slice_list = sorted(glob(f'../../data/raw_dicom/CT/{class_list[i]}/*'))
    os.makedirs(f'{save_path}images/{class_list[i]}', exist_ok=True)
    os.makedirs(f'{save_path}masks/{class_list[i]}', exist_ok=True)
    for j in tqdm(range(len(slice_list))):
        dcm_list = sorted(glob(f'{slice_list[j]}/*.dcm'))
        nii_name = os.path.basename(slice_list[j])

        volume_img = []
        volume_mask = []

        for dcm_file in dcm_list:
            dcm = pydicom.dcmread(dcm_file)
            img = dcm.pixel_array
            img_norm = window_and_normalize(img, dcm)

            # 이미지 전처리 + 마스크 추출
            gray = (img_norm * 127.5 + 127.5).astype(np.uint8)
            img_proc, mask = process_image(gray)
            img_proc = cv2.resize(img_proc, (128, 128), interpolation=cv2.INTER_LINEAR)
            volume_img.append(img_proc.astype(np.float32)/127.5-1)
            mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
            volume_mask.append((mask > 0).astype(np.uint8))

        # Stack 3D
        volume_img = np.stack(volume_img, axis=0)   # shape: (D, H, W)
        volume_mask = np.stack(volume_mask, axis=0) # shape: (D, H, W)

        # Save as NIfTI
        nii_img = nib.Nifti1Image(volume_img, affine=np.eye(4))
        nii_mask = nib.Nifti1Image(volume_mask, affine=np.eye(4))

        nib.save(nii_img, f'{save_path}images/{class_list[i]}/{nii_name}.nii.gz')
        nib.save(nii_mask, f'{save_path}masks/{class_list[i]}/{nii_name}.nii.gz')

100%|██████████| 474/474 [03:46<00:00,  2.09it/s]


In [4]:
'../../data/stroke_ct_nii/images/Ischemic/ANO1_0005.nii.gz'

'../../data/stroke_ct_nii/images/Ischemic/ANO1_0005.nii.gz'